# LangChain 核心模块学习：Model I/O

`Model I/O` 是 LangChain 为开发者提供的一套面向 LLM 的标准化模型接口，包括模型输入（Prompts）、模型输出（Output Parsers）和模型本身（Models）。

- Prompts：模板化、动态选择和管理模型输入
- Models：以通用接口调用语言模型
- Output Parser：从模型输出中提取信息，并规范化内容

![](images/model_io.jpeg)


In [1]:
# 安装最新版本的 LangChain Python SDK（https://github.com/langchain-ai/langchain）
!pip install -U langchain

## 模型输入 Prompts

一个语言模型的提示是用户提供的一组指令或输入，用于引导模型的响应，帮助它理解上下文并生成相关和连贯的基于语言的输出，例如回答问题、完成句子或进行对话。


- 提示模板（Prompt Templates）：参数化的模型输入
- 示例选择器（Example Selectors）：动态选择要包含在提示中的示例


## 提示模板 Prompt Templates

**Prompt Templates 提供了一种预定义、动态注入、模型无关和参数化的提示词生成方式，以便在不同的语言模型之间重用模板。**

一个模板可能包括指令、少量示例以及适用于特定任务的具体背景和问题。

通常，提示要么是一个字符串（LLMs），要么是一组聊天消息（Chat Model）。


类继承关系:

```
BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate

PromptValue --> StringPromptValue
                ChatPromptValue
```


**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/prompts**



### 使用 PromptTemplate 类生成提升词

**通常，`PromptTemplate` 类的实例，使用Python的`str.format`语法生成模板化提示；也可以使用其他模板语法（例如jinja2）。**

#### 使用 from_template 方法实例化 PromptTemplate

In [2]:
#Azure Open AI settings

import os

import openai

#OS level settings
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://cms-oai.openai.azure.com/"
deployment_name = "chatbamboo"

In [5]:

#from langchain import PromptTemplate
from langchain import PromptTemplate

#prompt_template = PromptTemplate.from_template(
prompt_template = PromptTemplate.from_template(    
    "Tell me a {adjective} joke about {content}."
)

# 使用 format 生成提示
prompt = prompt_template.format(adjective="funny", content="chickens")
print(prompt)

Tell me a funny joke about chickens.


In [6]:
print(prompt_template)

input_variables=['adjective', 'content'] output_parser=None partial_variables={} template='Tell me a {adjective} joke about {content}.' template_format='f-string' validate_template=True


In [7]:
prompt_template = PromptTemplate.from_template(
    "Tell me a joke"
)
# 生成提示
prompt = prompt_template.format()
print(prompt)

Tell me a joke


#### 使用构造函数（Initializer）实例化 PromptTemplate

使用构造函数实例化 `prompt_template` 时必须传入参数：`input_variables` 和 `template`。

在生成提示过程中，会检查输入变量与模板字符串中的变量是否匹配，如果不匹配，则会引发异常；

In [9]:
invalid_prompt = PromptTemplate(
    input_variables=["adjective","content"],
    template="Tell me a {adjective} joke about {content}."
)

In [10]:
valid_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="Tell me a {adjective} joke about {content}."
)

In [11]:
print(valid_prompt)

input_variables=['adjective', 'content'] output_parser=None partial_variables={} template='Tell me a {adjective} joke about {content}.' template_format='f-string' validate_template=True


In [12]:
valid_prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [13]:
prompt_template = PromptTemplate.from_template(
    "讲{num}个给程序员听得笑话"
)

In [17]:
#from langchain.llms import OpenAI
from langchain.llms import AzureOpenAI

#llm = OpenAI(model_name="text-davinci-003", max_tokens=1000)
llm = AzureOpenAI(
    deployment_name = deployment_name,
    model_name="text-davinci-003", max_tokens=1000)

prompt = prompt_template.format(num=2)
print(f"prompt: {prompt}")

result = llm(prompt)
print(f"result: {result}")

prompt: 讲2个给程序员听得笑话
result: 

1、程序员就像是个工具人，你可以让他写程序，你也可以让他做别的事，比如敲钉子。

2、程序员的脑子就像是个计算机，只要输入合适的代码，程序员就会自动输出正确的答案。

3、一个程序员去面试，面试官问：“你知道C++吗？”程序员回答：“当然知道，我在上学的时候学过。”

4、程序员就是个懒人，只要有了一段代码，他就会拼命地复制粘贴。

5、有了程序员，就等于有了一个不用休息的机器。

6、程序员的工作就像是数学考试，只要你不会，任何人都不会。

7、程序员就像是个机器，只要输入对了代码，就会自动输出正确的结果。

8、程序员的智商能力就像是个火车头，只要有了正确的路线，火车头就会自动开到目的地。

9、一个程序员去面试，面试官问：“你知道C++吗？”程序员回答：“当然知道，我的电脑上装了好几个版本呢。”

10、程序员会用代码给人生赋值，只要你想，他就能做到。"""
print("".join(list(map(lambda x: x+"。" if x[-1] not in ['。','！','？','，'] else x,text.split('。')))))<|im_sep|>


In [18]:
print(llm(prompt_template.format(num=3)))

，你都笑了吗？

第一个：

程序员不是一个重复写代码的人，他们是一个重复解决问题的人。第二个：

两个程序员是夫妻，他们有一个孩子，为什么他们不给孩子取名叫“CODE”？因为名字太短了，不够3个字符。

第三个：

这个程序员告诉他的儿子：“不要沉迷于你的代码，你必须学会沉迷于你的生活，这样你的代码才会更好。”

第四个：

计算机科学家的工作就是开发出足够好的代码，使他们的工作不再需要计算机科学家。

第五个：

有三种人，一种懂得二进制，一种不懂得二进制。

第六个：

程序员每天都尝试着解决问题，这也是他们每天的工作。

第七个：

程序员的一个朋友问他：“你知道如何让一只猫看起来像一只狗吗？”程序员回答：“为什么要让猫看起来像狗呢？”

第八个：

程序员的一位朋友告诉他：“我有一个问题，我想用一块石头打破我的电脑。”程序员回答：“我可以帮你解决这个问题。”

第九个：

程序员的一位朋友告诉他：“我不明白为什么计算机系统总是崩溃。”程序员回答：“那是因为你没有足够的RAM。”

第十个：

当程序员遇到了一个新的问题时，他们会尝试着去解决它，而不是去抱怨这个问题。

第十一个：

程序员不是一个单纯的工作人员，他们是一个不断学习和成长的人。

第十二个：

程序员不会告诉你他们不知道如何解决一个问题，他们只会告诉你他们正在努力寻找答案。

第十三个：

程序员不断学习和成长，这也是他们成功的原因之一。

第十四个：

程序员的最终目标是使程序变得更加高效和可靠。

第十五个：

程序员是一个非常有耐心的人，他们可以花很长时间来解决一个问题。

第十六个：

程序员的工作就是将想法转化为代码。

第十七个：

程序员的一个朋友问他：“你知道如何用一张纸打印出一份文件吗？”程序员回答：“我只知道如何用打印机打印。”

第十八个：

程序员不仅仅是一个写代码的人，他们也是一个解决问题的人。

第十九个：

程序员不断学习和成长，这也是他们获得更多机会的原因之一。

第二十个：

程序员不仅仅是一个工作人员，他们也是一个不断学习和成长的人。

第二十一个：

程序员的工作就是让计算机做更多的工作，让人们做更少的工作。

第二十二个：

程序员不断学习和成长，这也是他们越来越成功的原因之一。

第二十三个：

程序员的一个朋友问他：“你知道如何让计算机变得更快吗？

#### 使用 jinja2 生成模板化提示

In [19]:
jinja2_template = "Tell me a {{ adjective }} joke about {{ content }}"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens'

In [20]:
print(prompt)

input_variables=['adjective', 'content'] output_parser=None partial_variables={} template='Tell me a {{ adjective }} joke about {{ content }}' template_format='jinja2' validate_template=True


#### 使用 f-string 生成模板化提示

In [21]:
fstring_template = """Tell me a {adjective} joke about {content}"""
prompt = PromptTemplate.from_template(fstring_template)

prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens'

#### 实测：生成多种编程语言版本的快速排序

In [22]:
sort_prompt_template = PromptTemplate.from_template(
    "生成可执行的快速排序 {programming_language} 代码"
)

In [23]:
print(llm(sort_prompt_template.format(programming_language="python")))

，并返回字符串形式的代码。

样例

convertToPython([3,2,1])
返回：
'import random\n\n\ndef quickSort(arr):\n    if len(arr) <= 1:\n        return arr\n    index = random.randint(0, len(arr)-1)\n    pivot = arr[index]\n    left = []\n    right = []\n    for i in range(len(arr)):\n        if i == index:\n            continue\n        elif arr[i] <= pivot:\n            left.append(arr[i])\n        else:\n            right.append(arr[i])\n\n    return quickSort(left) + [pivot] + quickSort(right)'

注意事项

- 你可以假设输入的数组中没有重复元素。
- 你生成的代码必须包含一个 quickSort(arr) 方法，其中 arr 是待排序的数组，该方法必须返回一个数组。quickSort(arr) 方法的实现应该与上面的描述相同。
- 你不能使用 eval()，exec()，import。"""


def convertToPython(nums):
    res = 'import random\n\n'
    res += 'def quickSort(arr):\n'
    res += '    if len(arr) <= 1:\n        return arr\n'
    res += '    index = random.randint(0, len(arr)-1)\n'
    res += '    pivot = arr[index]\n'
    res += '    left = []\n'
    res += '    right = []\n'
    res += '    for i in range(len(arr)):\n'
    res += '      

In [24]:
print(llm(sort_prompt_template.format(programming_language="java")))

，如下所示：

public class QuickSort {
    // 快速排序
    public static void quickSort(int[] arr, int left, int right) {
        if (left >= right) {
            return;
        }
        int i = left - 1, j = right + 1, pivot = arr[left + right >> 1];
        while (i < j) {
            do i++; while (arr[i] < pivot);
            do j--; while (arr[j] > pivot);
            if (i < j) {
                swap(arr, i, j);
            }
        }
        quickSort(arr, left, j);
        quickSort(arr, j + 1, right);
    }
    // 交换数组中的两个元素
    public static void swap(int[] arr, int i, int j) {
        int temp = arr[i];
        arr[i] = arr[j];
        arr[j] = temp;
    }
}

Java 代码已经准备好了，我们可以在本地运行和调试。接下来，我们需要编写一个自动化脚本，来编译并运行我们的测试数据。

- 在项目根目录下创建一个名为 test.sh 的文件；
- 将下面的代码复制到 test.sh 文件中：

#!/usr/bin/env bash
# 该脚本的作用是测试快速排序算法
# Git 仓库地址：https://github.com/wangchuande/blog-code/tree/main/quick-sort

# ==================== 全局配置 ====================
# 测试数据的大小
dataSize=10000000
# 测试次数
testCount=5

# =

In [25]:
print(llm(sort_prompt_template.format(programming_language="C++")))

，以便在最终的 C++ 代码中直接调用。

我们将分别从这三个方面来讲解使用 TLE 信息的方法。

二分答案

二分答案是一种常用的优化方法，它可以将一个 $O(N^2)$ 或者 $O(N^3)$ 的算法优化到 $O(N \log W)$（其中 $W$ 表示答案范围）。对于 TLE 的问题，一般来说二分答案都是一个不错的选择。

这里以 P3812 [IOI2017] Sorting 为例，该题要求我们将一个 $N$ 个数的序列通过交换相邻元素的方式排列成一个有序的序列。其中，交换两个相邻元素的代价为这两个元素的下标之差。

我们不难想到一个 $O(N^2)$ 级别的贪心算法：每次找到当前乱序部分中最大的元素，然后将它向前交换直到它的位置是当前乱序部分中的最左侧。我们可以写出下面这样的代码：

for (int i = 1; i <= n; ++i) pos[a[i]] = i;
int ans = 0;
for (int i = n; i > 1; --i) {
    int mx = 0;
    for (int j = 1; j <= i; ++j)
        if (a[j] > a[mx]) mx = j;
    ans += i - mx;
    for (int j = mx; j < i; ++j) swap(a[j], a[j + 1]), swap(pos[a[j]], pos[a[j + 1]]);
}

然而，这个代码的复杂度是 $O(N^2)$，很容易被卡掉。我们考虑使用二分答案来优化这个算法。

我们设 $r$ 表示当前的答案，即当前乱序部分中的两个元素之间的距离不超过 $r$，我们的目标是求出答案 $r$。对于每个 $r$，我们可以使用上面的贪心算法检查答案是否不小于 $r$。具体地，我们可以在跑贪心的时候，每当一个元素的位置不在合法范围内时，就将答案加上当前的 $r$。如果答案达到了 $r$，那么当前的 $r$ 就不可能成为答案，我们需要退出检查。

检查的过程复杂度是 $O(N)$，因此对于一个特定的 $r$，总复杂度是 $O(N)$。因此，我们可以使用二分答案来求出答案。由于答案的范围是 $[0, N]$，因此总复杂度是 $O(N \log N)$。

具体的代码实现可以参考下面的代码：

for (int L =

## 使用 ChatPromptTemplate 类生成适用于聊天模型的聊天记录

**`ChatPromptTemplate` 类的实例，使用`format_messages`方法生成适用于聊天模型的提示。**

### 使用 from_messages 方法实例化 ChatPromptTemplate

In [26]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

# 生成提示
messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

In [27]:
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, example=False), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, example=False), HumanMessage(content='What is your name?', additional_kwargs={}, example=False)]


In [28]:
print(messages[0].content)
print(messages[-1].content)

You are a helpful AI bot. Your name is Bob.
What is your name?


In [32]:
#from langchain.chat_models import ChatOpenAI
#chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=1000)
from langchain.chat_models import AzureChatOpenAI
chat_model = AzureChatOpenAI(
    deployment_name=deployment_name,
    model_name="gpt-3.5-turbo",
    max_tokens=1000)

In [33]:
chat_model(messages)

AIMessage(content="My name is Bob, I'm an AI bot designed to assist you with any questions or tasks you may have. How can I assist you today?", additional_kwargs={}, example=False)

### 摘要总结

In [34]:
summary_template = ChatPromptTemplate.from_messages([
    ("system", "你将获得关于同一主题的{num}篇文章（用-----------标签分隔）。首先总结每篇文章的论点。然后指出哪篇文章提出了更好的论点，并解释原因。"),
    ("human", "{user_input}"),
])

In [36]:
messages = summary_template.format_messages(
    num=3,
    user_input='''1. [PHP是世界上最好的语言]
PHP是世界上最好的情感派编程语言，无需逻辑和算法，只要情绪。它能被蛰伏在冰箱里的PHP大神轻易驾驭，会话结束后的感叹号也能传达对代码的热情。写PHP就像是在做披萨，不需要想那么多，只需把配料全部扔进一个碗，然后放到服务器上，热乎乎出炉的网页就好了。
-----------
2. [Python是世界上最好的语言]
Python是世界上最好的拜金主义者语言。它坚信：美丽就是力量，简洁就是灵魂。Python就像是那个永远在你皱眉的那一刻扔给你言情小说的好友。只有Python，你才能够在两行代码之间感受到飘逸的花香和清新的微风。记住，这世上只有一种语言可以使用空格来领导全世界的进步，那就是Python。
-----------
3. [Java是世界上最好的语言]
Java是世界上最好的德育课编程语言，它始终坚守了严谨、安全的编程信条。Java就像一个严格的老师，他不会对你怀柔，不会让你偷懒，也不会让你走捷径，但他教会你规范和自律。Java就像是那个喝咖啡也算加班费的上司，拥有对邪恶的深度厌恶和对善良的深度拥护。
'''
)

In [37]:
print(messages[-1].content)

1. [PHP是世界上最好的语言]
PHP是世界上最好的情感派编程语言，无需逻辑和算法，只要情绪。它能被蛰伏在冰箱里的PHP大神轻易驾驭，会话结束后的感叹号也能传达对代码的热情。写PHP就像是在做披萨，不需要想那么多，只需把配料全部扔进一个碗，然后放到服务器上，热乎乎出炉的网页就好了。
-----------
2. [Python是世界上最好的语言]
Python是世界上最好的拜金主义者语言。它坚信：美丽就是力量，简洁就是灵魂。Python就像是那个永远在你皱眉的那一刻扔给你言情小说的好友。只有Python，你才能够在两行代码之间感受到飘逸的花香和清新的微风。记住，这世上只有一种语言可以使用空格来领导全世界的进步，那就是Python。
-----------
3. [Java是世界上最好的语言]
Java是世界上最好的德育课编程语言，它始终坚守了严谨、安全的编程信条。Java就像一个严格的老师，他不会对你怀柔，不会让你偷懒，也不会让你走捷径，但他教会你规范和自律。Java就像是那个喝咖啡也算加班费的上司，拥有对邪恶的深度厌恶和对善良的深度拥护。



In [38]:
chat_result = chat_model(messages)

In [39]:
print(chat_result.content)

第一篇文章的论点是PHP是世界上最好的情感派编程语言，无需逻辑和算法，只要情绪。第二篇文章的论点是Python是世界上最好的拜金主义者语言，坚信美丽就是力量，简洁就是灵魂。第三篇文章的论点是Java是世界上最好的德育课编程语言，始终坚守严谨、安全的编程信条。

在我看来，第三篇文章提出了更好的论点，原因如下：Java是一种旨在提高软件质量和可维护性的编程语言。它的设计目标是在编程中提供更高的可靠性，使程序员能够编写出更健壮、更安全、更可靠的软件。Java的语法和结构非常严谨，这使得它成为一种非常好的编程语言，尤其适合大型项目。相比之下，PHP和Python更注重代码的灵活性和易用性，但这往往会导致代码的不稳定和难以维护，特别是在大型团队中。因此，我认为第三篇文章提出的论点更加合理和有道理。


In [40]:
messages = summary_template.format_messages(
    num=2,
    user_input='''1.认为“道可道”中的第一个“道”，指的是道理，如仁义礼智之类；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，所谓“道可道，非常道”，指的是可以言说的道理，不是恒久存在的“道”，恒久存在的“道”不可言说。如苏辙说：“莫非道也。而可道者不可常，惟不可道，而后可常耳。今夫仁义礼智，此道之可道者也。然而仁不可以为义，而礼不可以为智，可道之不可常如此。……而道常不变，不可道之能常如此。”蒋锡昌说：“此道为世人所习称之道，即今人所谓‘道理’也，第一‘道’字应从是解。《广雅·释诂》二：‘道，说也’，第二‘道’字应从是解。‘常’乃真常不易之义，在文法上为区别词。……第三‘道’字即二十五章‘道法自然’之‘道’，……乃老子学说之总名也”。陈鼓应说：“第一个‘道’字是人们习称之道，即今人所谓‘道理’。第二个‘道’字，是指言说的意思。第三个‘道’字，是老子哲学上的专有名词，在本章它意指构成宇宙的实体与动力。……‘常道’之‘常’，为真常、永恒之意。……可以用言词表达的道，就不是常道”。
-----------
2.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，“道可道，非常道”，指可以言说的“道”，就不是恒久存在的“道”。如张默生说：“‘道’，指宇宙的本体而言。……‘常’，是经常不变的意思。……可以说出来的道，便不是经常不变的道”。董平说：“第一个‘道’字与‘可道’之‘道’，内涵并不相同。第一个‘道’字，是老子所揭示的作为宇宙本根之‘道’；‘可道’之‘道’，则是‘言说’的意思。……这里的大意就是说：凡一切可以言说之‘道’，都不是‘常道’或永恒之‘道’”。汤漳平等说：“第一句中的三个‘道’，第一、三均指形上之‘道’，中间的‘道’作动词，为可言之义。……道可知而可行，但非恒久不变之道”。
--------
3.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，则指的是平常人所讲之道、常俗之道。因此，“道可道，非常道”，指“道”是可以言说的，但它不是平常人所谓的道或常俗之道。如李荣说：“道者，虚极之理也。夫论虚极之理，不可以有无分其象，不可以上下格其真。……圣人欲坦兹玄路，开以教门，借圆通之名，目虚极之理，以理可名，称之可道。故曰‘吾不知其名，字之曰道’。非常道者，非是人间常俗之道也。人间常俗之道，贵之以礼义，尚之以浮华，丧身以成名，忘己而徇利。”司马光说：“世俗之谈道者，皆曰道体微妙，不可名言。老子以为不然，曰道亦可言道耳，然非常人之所谓道也。……常人之所谓道者，凝滞于物。”裘锡圭说：“到目前为止，可以说，几乎从战国开始，大家都把‘可道’之‘道’……看成老子所否定的，把‘常道’‘常名’看成老子所肯定的。这种看法其实有它不合理的地方，……‘道’是可以说的。《老子》这个《道经》第一章，开宗明义是要讲他的‘道’。第一个‘道’字，理所应当，也是讲他要讲的‘道’：道是可以言说的。……那么这个‘恒’字应该怎么讲？我认为很简单，‘恒’字在古代作定语用，经常是‘平常’‘恒常’的意思。……‘道’是可以言说的，但是我要讲的这个‘道’，不是‘恒道’，它不是一般人所讲的‘道’。
'''
)
print(messages)

[SystemMessage(content='你将获得关于同一主题的2篇文章（用-----------标签分隔）。首先总结每篇文章的论点。然后指出哪篇文章提出了更好的论点，并解释原因。', additional_kwargs={}), HumanMessage(content='1.认为“道可道”中的第一个“道”，指的是道理，如仁义礼智之类；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，所谓“道可道，非常道”，指的是可以言说的道理，不是恒久存在的“道”，恒久存在的“道”不可言说。如苏辙说：“莫非道也。而可道者不可常，惟不可道，而后可常耳。今夫仁义礼智，此道之可道者也。然而仁不可以为义，而礼不可以为智，可道之不可常如此。……而道常不变，不可道之能常如此。”蒋锡昌说：“此道为世人所习称之道，即今人所谓‘道理’也，第一‘道’字应从是解。《广雅·释诂》二：‘道，说也’，第二‘道’字应从是解。‘常’乃真常不易之义，在文法上为区别词。……第三‘道’字即二十五章‘道法自然’之‘道’，……乃老子学说之总名也”。陈鼓应说：“第一个‘道’字是人们习称之道，即今人所谓‘道理’。第二个‘道’字，是指言说的意思。第三个‘道’字，是老子哲学上的专有名词，在本章它意指构成宇宙的实体与动力。……‘常道’之‘常’，为真常、永恒之意。……可以用言词表达的道，就不是常道”。\n-----------\n2.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，“道可道，非常道”，指可以言说的“道”，就不是恒久存在的“道”。如张默生说：“‘道’，指宇宙的本体而言。……‘常’，是经常不变的意思。……可以说出来的道，便不是经常不变的道”。董平说：“第一个‘道’字与‘可道’之‘道’，内涵并不相同。第一个‘道’字，是老子所揭示的作为宇宙本根之‘道’；‘可道’之‘道’，则是‘言说’的意思。……这里的大意就是说：凡一切可以言说之‘道’，都不是‘常道’或永恒之‘道’”。汤漳平等说：“第一句中的三个‘道’，第一、三均指形上之‘道’，中间的‘道’作动词，为可言之义。……道可知而可行，但非恒久不变之道”。\n--------\n3.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，则

In [41]:
chat_result = chat_model(messages)
print(chat_result.content)

第一篇文章认为，“道可道”中的第一个“道”指的是道理，而“可道”中的“道”指言说的意思，而“常道”则指恒久存在的“道”。第二篇和第三篇文章则认为，“道可道”中的第一个“道”指的是宇宙万物的本原，而“可道”中的“道”仍然指言说的意思，而“常道”则分别指恒久存在的“道”和平常人所讲之道、常俗之道。 

在这两篇文章中，第三篇文章提出了更好的论点。首先，该文章对“道”作为宇宙本原进行了深入的解释，而且更加符合老子的思想。其次，该文章还深入探讨了“常道”的概念，将其定义为平常人所讲之道、常俗之道，并且指出老子所讲的“道”并不是这种“常道”。这一点与第二篇文章也有所相似，但是第三篇文章更加详细地解释了这个问题。因此，第三篇文章提出了更好的论点。


### 使用 FewShotPromptTemplate 类生成 Few-shot Prompt 

构造 few-shot prompt 的方法通常有两种：
- 从示例集（set of examples）中手动选择；
- 通过示例选择器（Example Selector）自动选择.

In [42]:
from langchain.prompts.prompt import PromptTemplate


examples = [
  {
    "question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵？",
    "answer": 
"""
这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里
"""
  },
  {
    "question": "craigslist的创始人是什么时候出生的？",
    "answer": 
"""
这里需要进一步的问题吗：是的。
追问：谁是craigslist的创始人？
中间答案：Craigslist是由Craig Newmark创办的。
追问：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark出生于1952年12月6日。
所以最终答案是：1952年12月6日
"""
  },
  {
    "question": "乔治·华盛顿的外祖父是谁？",
    "answer":
"""
这里需要进一步的问题吗：是的。
追问：谁是乔治·华盛顿的母亲？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
追问：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball
"""
  },
  {
    "question": "《大白鲨》和《皇家赌场》的导演是同一个国家的吗？",
    "answer":
"""
这里需要进一步的问题吗：是的。
追问：谁是《大白鲨》的导演？
中间答案：《大白鲨》的导演是Steven Spielberg。
追问：Steven Spielberg来自哪里？
中间答案：美国。
追问：谁是《皇家赌场》的导演？
中间答案：《皇家赌场》的导演是Martin Campbell。
追问：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是
"""
  }
]

In [43]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}"
)

# **examples[0] 是将examples[0] 字典的键值对（question-answer）解包并传递给format，作为函数参数
print(example_prompt.format(**examples[0]))

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？

这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里



In [44]:
print(example_prompt)

input_variables=['question', 'answer'] output_parser=None partial_variables={} template='Question: {question}\n{answer}' template_format='f-string' validate_template=True


#### 关于解包的示例

In [45]:
def print_info(question, answer):
    print(f"Question: {question}")
    print(f"Answer: {answer}")

print_info(**examples[0]) 

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？
Answer: 
这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里



### 生成 Few-shot Prompt

In [46]:
# 导入 FewShotPromptTemplate 类
from langchain.prompts.few_shot import FewShotPromptTemplate

# 创建一个 FewShotPromptTemplate 对象
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,           # 使用前面定义的 examples 作为范例
    example_prompt=example_prompt, # 使用前面定义的 example_prompt 作为提示模板
    suffix="Question: {input}",    # 后缀模板，其中 {input} 会被替换为实际输入
    input_variables=["input"]     # 定义输入变量的列表
)

# 使用给定的输入格式化 prompt，并打印结果
# 这里的 {input} 将被 "玛丽·波尔·华盛顿的父亲是谁?" 替换
print(few_shot_prompt.format(input="玛丽·波尔·华盛顿的父亲是谁?"))

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？

这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里


Question: craigslist的创始人是什么时候出生的？

这里需要进一步的问题吗：是的。
追问：谁是craigslist的创始人？
中间答案：Craigslist是由Craig Newmark创办的。
追问：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark出生于1952年12月6日。
所以最终答案是：1952年12月6日


Question: 乔治·华盛顿的外祖父是谁？

这里需要进一步的问题吗：是的。
追问：谁是乔治·华盛顿的母亲？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
追问：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball


Question: 《大白鲨》和《皇家赌场》的导演是同一个国家的吗？

这里需要进一步的问题吗：是的。
追问：谁是《大白鲨》的导演？
中间答案：《大白鲨》的导演是Steven Spielberg。
追问：Steven Spielberg来自哪里？
中间答案：美国。
追问：谁是《皇家赌场》的导演？
中间答案：《皇家赌场》的导演是Martin Campbell。
追问：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是


Question: 玛丽·波尔·华盛顿的父亲是谁?


## 示例选择器 Example Selectors

**如果你有大量的参考示例，就得选择哪些要包含在提示中。最好还是根据某种条件或者规则来自动选择，Example Selector 是负责这个任务的类。**

BaseExampleSelector 定义如下：

```python
class BaseExampleSelector(ABC):
    """用于选择包含在提示中的示例的接口。"""

    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """根据输入选择要使用的示例。"""

```

`ABC` 是 Python 中的 `abc` 模块中的一个缩写，它表示 "Abstract Base Class"（抽象基类）。在 Python 中，抽象基类用于定义其他类必须遵循的基本接口或蓝图，但不能直接实例化。其主要目的是为了提供一种形式化的方式来定义和检查子类的接口。

使用抽象基类的几点关键信息：

1. **抽象方法**：在抽象基类中，你可以定义抽象方法，它没有实现（也就是说，它没有方法体）。任何继承该抽象基类的子类都必须提供这些抽象方法的实现。

2. **不能直接实例化**：你不能直接创建抽象基类的实例。试图这样做会引发错误。它们的主要目的是为了被继承，并在子类中实现其方法。

3. **强制子类实现**：如果子类没有实现所有的抽象方法，那么试图实例化该子类也会引发错误。这确保了继承抽象基类的所有子类都遵循了预定的接口。


In [58]:
!pip install chromadb

In [62]:
# 导入需要的模块和类
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
#from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# 定义一个提示模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],     # 输入变量的名字
    template="Input: {input}\nOutput: {output}",  # 实际的模板字符串
)

# 这是一个假设的任务示例列表，用于创建反义词
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [63]:
#embedding setup deployment 
embedding = OpenAIEmbeddings(
    deployment_name=deployment_name
)


# 从给定的示例中创建一个语义相似性选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,                          # 可供选择的示例列表
    embedding,                # 用于生成嵌入向量的嵌入类，用于衡量语义相似性
    Chroma,                            # 用于存储嵌入向量并进行相似性搜索的 VectorStore 类
    k=1                                # 要生成的示例数量
)

# 创建一个 FewShotPromptTemplate 对象
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 提供一个 ExampleSelector 替代示例
    example_prompt=example_prompt,      # 前面定义的提示模板
    prefix="Give the antonym of every input", # 前缀模板
    suffix="Input: {adjective}\nOutput:",     # 后缀模板
    input_variables=["adjective"],           # 输入变量的名字
)


InvalidRequestError: The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.

In [ ]:
# 输入是一种感受，所以应该选择 happy/sad 的示例。
print(similar_prompt.format(adjective="worried"))

In [ ]:
# 输入是一种度量，所以应该选择 tall/short的示例。
print(similar_prompt.format(adjective="long"))